## Class work wiht STT

In [1]:
!pip3 install whisper-timestamped
!pip install torch
!pip install moviepy

In [1]:
import whisper_timestamped as whisper     # For convert to text, STT (Speach to text)
import torch                              # For whisper
from moviepy.editor import VideoFileClip  # For convert MP4 to MP3
import uuid                               # For generate id to user video data
import pickle                             # Save user video data on file
import os                                 # Work with file

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [34]:
class STT:
    def __init__(self, modelType="base"):
        self.modelType = modelType
        self.batch_size = 16 # reduce if low on GPU mem
        self.divice = "cuda" if torch.cuda.is_available() else "cpu"
        self.compute_type = "float16" if torch.cuda.is_available() else "int8"
        self.model = whisper.load_model(self.modelType, device=self.divice)

    def convertMP3ToText(self, pathToMP3: str) -> (str, str):
        audio = whisper.load_audio(pathToMP3)
        result = whisper.transcribe(self.model, audio, language="ru")

        text: str = result["text"]
        word_data: list[dict] = list()
        for segment in result["segments"]:
            for word in segment["words"]:
                word_data.append({
                    "text": word["text"],
                    "startTime": word["start"] * 1000,
                    "endTime": word["end"] * 1000
                })
        return text, word_data

    def saveData(self, text: str, word_data: str) -> str:
        absPath = os.path.abspath("")
        
        # Saving data
        saveData = {
            "text": text,
            "word_data": word_data
        }
        
        # Setting save directory
        pathDict = f"{absPath}/user"
        if not os.path.exists(pathDict):
            os.makedirs(pathDict)

        # Create id user video
        _id = uuid.uuid4().hex
        while os.path.isfile(f"{pathDict}/{_id}.pickle"):
            _id = uuid.uuid4().hex
    
        # Save data in pickle
        with open(f"{pathDict}/{_id}.pickle", 'wb') as file:
            pickle.dump(saveData, file, protocol=pickle.HIGHEST_PROTOCOL)
        return _id

    def loadData(self, _id: str) -> (str, list[dict]):
        absPath = os.path.abspath("")
        pathDict = f"{absPath}/user"
        
        # Check created file 
        if not os.path.isfile(f"{pathDict}/{_id}.pickle"):
            print("File does not exist")
            return "", list()

        # Open file
        with open(f"{pathDict}/{_id}.pickle", 'rb') as file:
            data = pickle.load(file)

        return data["text"], data["word_data"]

    def convertMP3(self, pathToMP3: str) -> str:
        text, word_data = self.convertMP3ToText(pathToMP3)
        _id = self.saveData(text, word_data)
        return _id

    def convertMP4(self, pahtToMP4: str) -> str:
        absPath = os.path.abspath("")
        pathTempMP3 = f"{absPath}/temp/mp3"
        fileName = os.path.splitext(os.path.basename(pahtToMP4))[0]
        tempFilePath = f"{pathTempMP3}/{fileName}.mp3"

        # Setting temp directory
        if not os.path.exists(pathTempMP3):
            os.makedirs(pathTempMP3)
        
        # Convert MP4 to MP3
        video = VideoFileClip(pahtToMP4)
        video.audio.write_audiofile(tempFilePath, logger=None)
        
        # Convert MP3 to data text
        _id = self.convertMP3(tempFilePath)

        # Delete temp file MP3
        if os.path.exists(tempFilePath):
            os.remove(tempFilePath)
        
        return _id

In [35]:
converter = STT("base")

In [36]:
test_audio = "/Users/odner/GitHub/hack20_09_27/viral_video/test_audio/test.mp3"

In [37]:
test_video = "/Users/odner/GitHub/hack20_09_27/viral_video/test/1c6bc481dd52a9938e78e755f1e5c90e.mp4"

In [38]:
converter.convertMP3(test_audio)



  0%|                                               | 0/874 [00:00<?, ?frames/s]

100%|███████████████████████████████████| 874/874 [00:00<00:00, 1608.44frames/s]


'7c227ab1edef4fc1af5b7e972fb326b2'

In [39]:
converter.loadData("7c227ab1edef4fc1af5b7e972fb326b2")

(' Привет! Как дела? Hello! Бондиур!',
 [{'text': 'Привет!', 'startTime': 1600.0, 'endTime': 1960.0},
  {'text': 'Как', 'startTime': 2760.0, 'endTime': 3140.0},
  {'text': 'дела?', 'startTime': 3140.0, 'endTime': 3460.0},
  {'text': 'Hello!', 'startTime': 4700.0, 'endTime': 5000.0},
  {'text': 'Бондиур!', 'startTime': 7040.0, 'endTime': 7700.0}])

In [40]:
converter.convertMP4(test_video)



  0%|                                             | 0/65422 [00:00<?, ?frames/s]

  4%|█▎                              | 2784/65422 [00:02<01:00, 1040.69frames/s]

  9%|██▊                             | 5664/65422 [00:04<00:43, 1383.10frames/s]

 13%|████▏                           | 8596/65422 [00:06<00:44, 1269.74frames/s]

 17%|█████▎                         | 11224/65422 [00:09<00:45, 1187.94frames/s]

 21%|██████▌                        | 13964/65422 [00:11<00:45, 1139.97frames/s]

 24%|███████▌                        | 15580/65422 [00:15<00:56, 886.32frames/s]

 27%|████████▊                       | 17980/65422 [00:18<00:56, 834.00frames/s]

 32%|██████████▏                     | 20880/65422 [00:20<00:46, 967.54frames/s]

 36%|███████████▌                    | 23580/65422 [00:23<00:45, 925.88frames/s]

 40%|████████████▉                   | 26480/65422 [00:26<00:41, 935.03frames/s]

 45%|██████████████▎                 | 29380/65422 [00:29<00:38, 929.18frames/s]

 49%|█████████

'8f04636c19a0424c9778aee2fc6517a3'

In [41]:
converter.loadData("8f04636c19a0424c9778aee2fc6517a3")

(' Наташа, ты эту кухню видела? Это 90. Я удивляюсь, как там я тут еще вот эту, значит, уголка мягкого. Ватору, в сиденье поднимаются и там вот эти пакеты, пакеты, крышки для консервации. Чем пакет-то сразу? А картошка? С морковкой. Ты хозяйку квартиры видела? Вот она. Наташа, по-моему, она все сдует, желание. Чем по-не придется услышать это еще раз. Пойдем, неудобно. Денька получилась, пошли. Это шоу о том, как сделать ремонт без ремонта? Как своими силами преобразить квартиру без бригады рабочих, без пыли и без большого бюджесса. Использует только декорг и дизайнерский лайфхаки. Так, идем идем. Ну, лен. Прости нас, пожалуйста, мы не то, чтобы там сплеть, мы тебя в лицо можем сказать. Я все слышала. Понимаешь, ты все слышала, ну кухня тебя правда, старая блячская. Ты молодая, кухня. Ну, она нас для этого сюда и позвала. Ничего не переживая. Мы что-нибудь придумаем, что не устраивает здесь. Ну, мне не особо нравится мой стол. Сразу замечу, вот эти салфетницы, они предназначены исключит

In [33]:
del converter